<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_4_ORPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 3 with ORPO


In [1]:
!pip install -qqq -U transformers datasets accelerate peft trl bitsandbytes wandb --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
import gc
import os
import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
import logging

### Colab
This part is only relevant when using the notebook in google colab

In [3]:
from google.colab import userdata, drive

In [4]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Mounted at /content/drive


In [5]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 1784, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 1784 (delta 212), reused 168 (delta 155), pack-reused 1530 (from 1)
Receiving objects: 100% (1784/1784), 51.37 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (1066/1066), done.


In [6]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 5
BASE_MODEL_REPO = "meta-llama/Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct_ORPO_1"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/NLP2025_CQG/Data/Processed/CQ DPO Dataset.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)


checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [7]:
logger.info("--------  Start with ORPO Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

INFO:__main__:--------  Start with ORPO Training  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Model: Meta-Llama-3.1-8B-Instruct_ORPO_1
INFO:__main__:Training number: 5


In [8]:
# Defined in the secrets tab in Google Colab
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)
torch_dtype = torch.bfloat16

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ricostaedeli (ricostaedeli-hsg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [20]:
dataset = load_dataset('json', data_files=train_dataset_path)
dataset = dataset.filter(lambda x: x['score_chosen'] - x['score_rejected'] > 4)

In [21]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected', 'schema', 'context'],
        num_rows: 1572
    })
})


In [22]:
def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    row["prompt"] = tokenizer.apply_chat_template(row["prompt"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

Map (num_proc=12):   0%|          | 0/1572 [00:00<?, ? examples/s]

In [23]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected', 'schema', 'context'],
        num_rows: 1572
    })
})


In [24]:
columns_to_remove = ['id', 'score_chosen', 'score_rejected', 'schema', 'context']

dataset = dataset['train'].remove_columns(columns_to_remove)

In [25]:
dataset = dataset.train_test_split(test_size=0.05)

In [26]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1493
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 79
    })
})


In [28]:
print("---------------   Prompt -----------------")
print(dataset['train'][0]['prompt'])
print("------------------------------------------")
print("---------------   Chosen -----------------")
print("------------------------------------------")
print(dataset['train'][0]['chosen'])
print("------------------------------------------")
print("---------------   Rejected -----------------")
print("------------------------------------------")
print(dataset['train'][0]['rejected'])

---------------   Prompt -----------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Generate one critical question addressing the provided context. Ensure it matches the schema: FearAppeal

Context: reasons_evidence: It is kind of messed up according to todays morality but morals change. I should have stated in the post that it was more of a thought exercise into a future moral shift. What's the point of life if your just working for nothing, you can't have kids or family? That line of inquiry will get us onto a whole load of philosophical topics. Just as a side note: My own personal opinion on that matter is that children are a poor (but hugely popular) purpose for life, all it does it push the issue on a generation where your children live for their children and so on and so on to no real end.<|eot_id|>
------------------------------------------
-----

## Start training

In [29]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    beta=0.1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    remove_unused_columns=False,
    optim="paged_adamw_8bit",
    max_steps = 40,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    processing_class=tokenizer,
)
trainer.train()
trainer.save_model(MODEL_NAME)

Map:   0%|          | 0/1493 [00:00<?, ? examples/s]

Map:   0%|          | 0/1493 [00:00<?, ? examples/s]

Map:   0%|          | 0/1493 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
8,16.145200,4.051310,37.487300,2.107000,1.067000,-0.343002,-0.265018,0.012500,-0.077984,-2.650177,-3.430016,-1.033975,-1.021217,3.932078,-1.202248,-0.821663
16,15.451200,3.704513,37.538400,2.105000,1.066000,-0.327893,-0.255134,0.012500,-0.072759,-2.551338,-3.278929,-1.074436,-1.065022,3.587935,-1.166939,-0.771834
24,13.765600,3.455454,37.521400,2.105000,1.066000,-0.310681,-0.242968,0.025000,-0.067714,-2.429678,-3.106814,-1.113497,-1.108522,3.341541,-1.134229,-0.725030
32,13.307000,3.311141,37.529600,2.105000,1.066000,-0.293772,-0.230611,0.037500,-0.063160,-2.306114,-2.937717,-1.151165,-1.148278,3.199807,-1.105796,-0.683675
40,12.650600,3.255154,37.524500,2.105000,1.066000,-0.286701,-0.225350,0.037500,-0.061351,-2.253498,-2.867007,-1.158781,-1.156559,3.144835,-1.094824,-0.667494


In [30]:
# Flush memory
#del trainer, model
gc.collect()
gc.collect()
torch.cuda.empty_cache()

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)

fp16_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, MODEL_NAME)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [31]:
model.push_to_hub(MODEL_NAME, use_temp_dir=False)
tokenizer.push_to_hub(MODEL_NAME, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ricostaedeli/Meta-Llama-3.1-8B-Instruct_ORPO_1/commit/702496e526b27d382780f58f66a2c39a8987a348', commit_message='Upload tokenizer', commit_description='', oid='702496e526b27d382780f58f66a2c39a8987a348', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ricostaedeli/Meta-Llama-3.1-8B-Instruct_ORPO_1', endpoint='https://huggingface.co', repo_type='model', repo_id='ricostaedeli/Meta-Llama-3.1-8B-Instruct_ORPO_1'), pr_revision=None, pr_num=None)

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive"
!git add .
!git commit -m "{commit_message}"
!git push